# FIRA FEINA VIRTUAL UOC 2021
## GERARD COSTA FIGUEROLA - Màster en Data Science

### Obertura i primera exploració

In [1]:
import numpy as np
import pandas as pd

#Obrim el train dataset
df = pd.read_csv('data/uoc_train.csv')



In [2]:
#Comprovem que no hi hagi valors nuls i els tipus de dades de cada variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   feature1  2100 non-null   float64
 1   feature2  2100 non-null   float64
 2   feature3  2100 non-null   float64
 3   feature4  2100 non-null   float64
 4   feature5  2100 non-null   float64
 5   feature6  2100 non-null   float64
 6   feature7  2100 non-null   float64
 7   feature8  2100 non-null   float64
 8   target    2100 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 147.8 KB


In [3]:
#Mirem quins valors i quants pot tenir la variable original 'target'
df['target'].value_counts()

0    713
1    705
2    682
Name: target, dtype: int64

### Divisió del dataset en X i y i en train i test

In [4]:
#Divideixo en arrays de X i Y el dataframe
X = df.values
#A la X li treiem la columna 'target' (que és la 8a)
X = np.delete(X, 8, axis = 1)

#I a la Y li deixem només la columna 'target'
y = df['target'].values

In [5]:
#Ara dividirem el dataset en train i test per tal de construir el classificador 
#Random Forest
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state = 2)


### Comprovació de les mides dels datasets i com de balancejats estan

In [6]:
#Imprimim les mides dels dos datasets i mirem com de balancejat està respecte l'original
print('Mida del train dataset: {}'.format(len(X_train)))
print('Mida del test dataset: {}'.format(len(X_test)))


#Per tal que surti un df prou equilibrat, he anat canviant el 
#random_state al train_test_split anterior
unique, counts = np.unique(y_train, return_counts=True)
print('\nTrain dataset')
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print('\nTest dataset')
print(dict(zip(unique, counts)))


Mida del train dataset: 1470
Mida del test dataset: 630

Train dataset
{0: 499, 1: 490, 2: 481}

Test dataset
{0: 214, 1: 215, 2: 201}


### Construcció i entrenament del model classificador RF v1

In [8]:
#Ara construim el classificador
from sklearn import ensemble

rf_clf = ensemble.RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state = 2)

#Entrenem al classificador
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=2)

### Generació de prediccions i comprovació de f1 score (macro)

In [35]:
#Generem un vector de les prediccions que fa el classificador amb el test dataset
from sklearn.metrics import f1_score

y_pred = rf_clf.predict(X_test)

f1_v1 = f1_score(y_test, y_pred, average='macro')
#I mirem quina f1 score (macro) té
print('F1 score (macro): {}'.format(f1_v1))

F1 score (macro): 0.9233299526707235


### Intent de millorar el model 1 (Hyperparameter tunning)
#### Creació de la grid de paràmatres

In [16]:
#Està molt bé, però mirem si podem millorar-la fent Hyperparameter tunning amb RandomizedSearchCV

#Primer hem de crear la parameter grid per donar-li al gridsearch

#Creem diferents vectors amb els diferents valors que volem provar de cada paràmetre
n_estimators = np.arange(50, 1200, 100)

max_features = ['auto', 'sqrt']

max_depth = [2, 4, None]

min_samples_split = [2, 3, 5]

min_samples_leaf = [1,2,3]

max_samples = [0.5, 0.75, 1]

bootstrap = [True, False]


#Creem un diccionari amb tots els paràmetres

param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'max_samples': max_samples,
              'bootstrap': bootstrap
}


#### Entrenament de la grid per buscar els millors paràmetres

In [33]:
#Creem el RandomizedSearchCV per veure quins són els millors paràmetres
#Aquest, provarà, de totes les combinacions possibles, 100 d'aleatòries i 
#després podrem saber quina té els millors resultats

from sklearn.model_selection import RandomizedSearchCV

#Ho farem mitjançant una funció:

#Aquesta funció entrenarà la xarxa provant un nombre determinat de combinacions random
#A partir de la xarxa de paràmetres que li passis.
#Retornarà un diccionari amb els millors paràmetres

def millors_param(classificador, param_grid):
    rf_grid = RandomizedSearchCV(estimator = classificador, 
                             param_distributions = param_grid,
                             cv = 10,
                             verbose = 0,
                             n_jobs = -1)
    print('Entrenant grid........ (aprox: 2m30s)')
    rf_grid.fit(X_train, y_train)
    return rf_grid.best_params_
    #(Tarda una estona ja que ha de provar unes quantes combinacions random de paràmetres)


#### Construcció i entrenament del model amb millors paràmetres segons RandomizedSearchCV

In [31]:
#Ho farem també amb una funció, que retornarà el nou model, la nova y_pred
#i imprimirà i retornarpa la f1 score segons el nou model ambels millors paràmetres 
#segons RandomizedSearchCV (per veure si millora)

from sklearn.metrics import f1_score

def model_best_param(classificador_original, param_grid, X_train, y_train, X_test, y_test):
    best_param = millors_param(classificador_original, param_grid)
    print('\nConstruint model amb els paràmetres:')
    print(best_param)
    
    rf_clf_tunned = ensemble.RandomForestClassifier(n_estimators= best_param['n_estimators'], 
                                                    n_jobs= -1, 
                                                    random_state = 2,
                                                    min_samples_split= best_param['min_samples_split'],
                                                    min_samples_leaf= best_param['min_samples_leaf'], 
                                                    max_samples= best_param['max_samples'], 
                                                    max_features=best_param['max_features'], 
                                                    max_depth= best_param['max_depth'],
                                                    bootstrap= best_param['bootstrap'])
    rf_clf_tunned.fit(X_train, y_train)
    
    y_pred_tunned = rf_clf_tunned.predict(X_test)
    f1 = f1_score(y_test, y_pred_tunned, average='macro')
    print('\nF1 score (macro): {}'.format(f1))
    return [rf_clf_tunned, y_pred_tunned, f1]
    
    

#### Prova de la funció per buscar el millors paràmetres

In [34]:
#Volem veure doncs si la f1 score millora:

rf_clf_v2 = model_best_param(rf_clf, param_grid, X_train, y_train, X_test, y_test)

Entrenant grid........ (aprox: 2m30s)

Construint model amb els paràmetres:
{'n_estimators': 750, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_samples': 0.75, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}

F1 score (macro): 0.9170614697431314


Provant-ho diferents vegades amb diferents paràmetres random no aconsegueixo millorar la F1 score incial de 0.92 (en la v2 sempre surt una F1 del voltant de 0.91) 

És probable doncs que no estigui definint molt bé la parameter grid...

### Intent de millorar el model 2 (Treure variables poc importants)

#### Comprovar la importància de cada variable

In [38]:
#Per intentar millorar el model, anem a veure quina és la importància de cada variable
#a l'hora de classificar-ho amb el primer model, l'original (que tenia una f1 una mica més alta)

cols = df.drop(['target'],1).columns
feature_importance_all = rf_clf.feature_importances_
feature_importance = {}
for i in  range(len(cols)):
    feature_importance[cols[i]] = feature_importance_all[i]
    
feature_import =  pd.DataFrame.from_dict(feature_importance, 
                                         orient = 'index')
print(feature_import)

#Veiem que les que més pesen són la feature 3 i la feature6
#I les que menys pesen la feature7 i la feature8

                 0
feature1  0.125156
feature2  0.162800
feature3  0.230482
feature4  0.074054
feature5  0.115082
feature6  0.234092
feature7  0.029971
feature8  0.028363


#### Tornant a separar en train, test, X i y però treient les dues variables menys importants

In [39]:
#La 7 i la 8 no tenen valors molt baixos però per provar si la f1 millora,
# les treurem del df train i test i tornarem a provar el model.


X2 = df.values
#A la X li treiem la columna 'target' (que és la 8a)
X2 = np.delete(X2, [6,7,8], axis = 1)

#I a la Y li deixem només la columna 'target'
y2 = df['target'].values

#Tornem a crear els datasets de train i test
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2, test_size=0.3, random_state = 2)

#### Tornem a crear, entrenar i mostrar la nova f1 amb el nou classificador v3

In [40]:
#Tornem a crear al classificador
rf_clf_v3 = ensemble.RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state = 2)

#Entrenem al classificador
rf_clf_v3.fit(X2_train, y2_train)

#El vector de prediccions
y_pred2 = rf_clf_v3.predict(X2_test)

f1_v3 = f1_score(y2_test, y_pred2, average='macro')

#I mirem quina f1 score (macro) té:
print('F1 score (macro): {}'.format(f1_v3))

#Veiem que té (quasi) la mateixa puntuació, augmenta només en 0.002 punts percentuals

F1 score (macro): 0.9250419331546912


#### Resumint, tenim 3 models, 
- El primer sense tocar res 
- El segon amb els paràmetres modificats 
- I el tercer treient les dues variables que menys importància tenen

In [42]:
   
print('F1 del primer model: {}'.format(f1_v1))
print('F1 del segon model (tunejat): {}'.format(rf_clf_v2[2]))
print('F1 del tercer model (sense les dues variables menys importants): {}'.format(f1_v3))



#Per poc que sigui, ens quedem doncs amb aquest últim model

F1 del primer model: 0.9233299526707235
F1 del segon model (tunejat): 0.9170614697431314
F1 del tercer model (sense les dues variables menys importants): 0.9250419331546912


### Per tant, ens quedem amb el *3r model*, que té la puntuació f1 més alta

### Aplicació del 3r model Random Forest al dataframe uoc_X_test

In [45]:
#I ara anem a aplicar aquest útlim model dataframe uoc_X_test
import pandas as pd
uoc_X_test = pd.read_csv('data/uoc_X_test.csv')

#Generem els valors de X, com que utilitzem l'últim model, haurem de treure les dues últimes features.
final_X = uoc_X_test.values
final_X = np.delete(final_X, [6,7], axis = 1)

#Generem els valors predits
final_status = rf_clf_v3.predict(final_X)

#Els posem en un dataframe
final_df = pd.DataFrame(final_status, columns = ['final_status'])

#I generem l'arxiu csv final
final_df.to_csv('predictions.csv',index=False)